# NDEx2 Tutorial

In this tutorial, we will first show you how to install the NDEx NiceCX module and use it to create a basic graph and to access one from the NDEx server. NiceCX requires Python 2.7.9 and the latest version of the PIP Python package manager for installation. For further details on installing the NDEx module, see the NDEx Client Tutorial.

## Importing Packages

With the NDEx module installed, start Python and import NDExGraph.

In [1]:
from nicecxModel.NiceCXNetwork import NiceCXNetwork
from nicecxModel.cx.aspects.NodesElement import NodesElement
from nicecxModel.cx.aspects.EdgesElement import EdgesElement
from nicecxModel.cx.aspects.NodeAttributesElement import NodeAttributesElement
from nicecxModel.cx.aspects.EdgeAttributesElement import EdgeAttributesElement
import networkx as nx
import pandas as pd

## Creating a NiceCX model

There are generally five ways to create a NiceCX object.
1. An empty object. 
          niceCx = NiceCXNetwork()
2. Using a cx dictionary. 
          niceCx = NiceCXNetwork(cx=cx)
3. Using networkx.
          niceCx = NiceCXNetwork(networkx_G=G)
4. Using a Pandas DataFrame.
          niceCx = NiceCXNetwork(pandas_df=df)
5. Loading it from an NDEx server.
          niceCx = NiceCXNetwork(server='test.ndexbio.org', uuid='01c83ba5-0d90-11e6-b550-06603eb7f303')


To access a private network, you will need to include your credentials- your username and password. Otherwise, just include a server and an uuid.

In [2]:
uuidTest= "01c83ba5-0d90-11e6-b550-06603eb7f303"

In [3]:
#Fifth option for creating a graph
niceCx = NiceCXNetwork(server='test.ndexbio.org', uuid='01c83ba5-0d90-11e6-b550-06603eb7f303')

http://test.ndexbio.org/v2/network/01c83ba5-0d90-11e6-b550-06603eb7f303/aspect


The second option will be covered towards the end of this tutorial. In this tutorial, we will use an empty graph to create a simple food web.

We will call the graph niceCx. The nodes will contain the color of the animal/plant and the edges will contain the way that they are related.

In [4]:
#First option for creating a graph
niceCx = NiceCXNetwork()

### Naming the Graph

You can set the name of the graph by using "set_name(graph_name)". In order to get the name, you can use "get_name()".

In [5]:
niceCx.setName("Food Web")

In [6]:
niceCx.getName()

'Food Web'

### Filling the Graph With Data

ndex2 is designed to be a simple data model.  To add a node to the network you need to build the NodesElement() object and then add it to the niceCx data model.  

In [7]:
niceCx = NiceCXNetwork()

#add_this_node_fox = NodesElement(node_name='Fox')
niceCx.addNode(node_name='Fox')

'Fox'

OR - You can combine the two lines into one to shorten the code

In [5]:
niceCx = NiceCXNetwork()

fox_node = niceCx.addNode(node_name='Fox')
mouse_node = niceCx.addNode(node_name='Mouse')
bird_node = niceCx.addNode(node_name='Bird')

fox_bird_edge = niceCx.addEdge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = niceCx.addEdge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')


The edges and nodes that you create will have a unique ID in the order that they appear. For example, the node with the name "Mouse" will have an ID of 0, "Fox" will have 1, and so on. The edge between "Fox" and "Mouse" will have the ID of 0, the next will be 1, and so on. Keep this in mind, because you will need these ID to access information about these nodes and edges. 

In order to add an attribute to a node already declared, use the NodeAttributesElement() object

In [6]:
niceCx.addNodeAttribute(property_of=fox_node, name='Color', values='Red')

niceCx.addNodeAttribute(property_of=mouse_node, name='Color', values='Gray')

niceCx.addNodeAttribute(property_of=bird_node, name='Color', values='Blue')

To set the edge's attribute, use "addEdgeAttribute()"

In [7]:
niceCx.addEdgeAttribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

In [8]:
print niceCx

[{"numberVerification": [{"longNumber": 281474976710655}]}, {"nodes": [{"@id": 0, "n": "Mouse"}, {"@id": 1, "n": "Fox"}, {"@id": 2, "n": "Bird"}]}, {"edges": [{"i": "interacts-with", "s": 1, "@id": 0, "t": 2}, {"i": "interacts-with", "s": 1, "@id": 1, "t": 0}]}, {"nodeAttributes": [{"v": "Gray", "po": 0, "n": "Color"}, {"v": "Red", "po": 1, "n": "Color"}, {"v": "Blue", "po": 2, "n": "Color"}]}, {"edgeAttributes": [{"v": "On the ground", "po": 1, "n": "Hunted"}]}, {"metaData": [{"elementCount": 3, "version": "1.0", "consistencyGroup": 1, "name": "nodeAttributes"}, {"idCounter": 2, "elementCount": 3, "version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"idCounter": 1, "elementCount": 2, "version": "1.0", "consistencyGroup": 1, "name": "edges"}, {"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "edgeAttributes"}]}]


## Create using CX dictionary

If you have a CX formatted dictionary you can use it to create a NiceCx model

In [ ]:
my_cx = [
    {"numberVerification":[{"longNumber":281474976710655}]},
    {"metaData":[{"consistencyGroup":1,"elementCount":2,"idCounter":2,"name":"nodes","version":"1.0"},
    {"consistencyGroup":1,"elementCount":1,"idCounter":1,"name":"edges","version":"1.0"}]},
    {"nodes":[{"@id": 1, "n": "node1", "r": "ABC"}, {"@id": 2, "n": "node2", "r": "DEF"}]},
    {"edges":[{"@id": 1, "s": 1, "t": 2, "i": "neighbor"}]},
    {"status":[{"error":"","success":True}]}
]

niceCx = NiceCXNetwork(cx=my_cx)


## Create using Networkx

The NiceCx model can be created using networkx as input

In [10]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_edges_from([('ABC','DEF')])

#====================================
# BUILD NICECX FROM NETWORKX GRAPH
#====================================
niceCx = NiceCXNetwork(networkx_G=G)
                       
print(niceCx)

[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"version": "1.0", "consistencyGroup": 1, "name": "edges"}]}, {"nodes": [{"r": "ABC", "@id": 0, "n": "ABC"}, {"r": "DEF", "@id": 1, "n": "DEF"}]}, {"edges": [{"s": 0, "@id": 0, "t": 1}]}, {"networkAttributes": [{"v": "Networkx Upload", "d": "string", "n": "name"}]}, {"metaData": [{"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"idCounter": 1, "elementCount": 2, "version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "edges"}]}]


# Create using Pandas DataFrame

Create a NiceCx model using Pandas DataFrame

### Using a 2 column dataframe with no headers:

In [12]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

#==============================================
# BUILD NICECX FROM PANDAS DATAFRAME 2-Column
#==============================================
niceCx = NiceCXNetwork(pandas_df=df)

print(niceCx)

[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"version": "1.0", "consistencyGroup": 1, "name": "edges"}]}, {"nodes": [{"r": "XYZ", "@id": 0, "n": "XYZ"}, {"r": "ABC", "@id": 1, "n": "ABC"}, {"r": "DEF", "@id": 2, "n": "DEF"}]}, {"edges": [{"i": "interacts-with", "s": 1, "@id": 0, "t": 2}, {"i": "interacts-with", "s": 2, "@id": 1, "t": 0}]}, {"networkAttributes": [{"v": "Pandas Upload", "d": "string", "n": "name"}]}, {"metaData": [{"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"idCounter": 2, "elementCount": 3, "version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"idCounter": 1, "elementCount": 2, "version": "1.0", "consistencyGroup": 1, "name": "edges"}]}]


### Using a 3 column dataframe with no headers:

In [5]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

#==============================================
# BUILD NICECX FROM PANDAS DATAFRAME 3-Column
#==============================================
niceCx = NiceCXNetwork(pandas_df=df)

print(niceCx)

[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"version": "1.0", "consistencyGroup": 1, "name": "edges"}]}, {"nodes": [{"r": "XYZ", "@id": 0, "n": "XYZ"}, {"r": "ABC", "@id": 1, "n": "ABC"}, {"r": "DEF", "@id": 2, "n": "DEF"}]}, {"edges": [{"i": "interacts-with", "s": 1, "@id": 0, "t": 2}, {"i": "neighbor-of", "s": 2, "@id": 1, "t": 0}]}, {"networkAttributes": [{"v": "Pandas Upload", "d": "string", "n": "name"}]}, {"metaData": [{"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"idCounter": 2, "elementCount": 3, "version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"idCounter": 1, "elementCount": 2, "version": "1.0", "consistencyGroup": 1, "name": "edges"}]}]


### Using 3+ columns plus headers to specify attributes columns

In [2]:
df = pd.DataFrame.from_items([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

niceCx = NiceCXNetwork()
#==================================================
# BUILD NICECX FROM PANDAS DATAFRAME WITH HEADERS
#==================================================
niceCx.create_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

print(niceCx)

0
[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"version": "1.0", "consistencyGroup": 1, "name": "edges"}, {"version": "1.0", "consistencyGroup": 1, "name": "edgeAttributes"}]}, {"nodes": [{"r": "XYZ", "@id": 0, "n": "XYZ"}, {"r": "ABC", "@id": 1, "n": "ABC"}, {"r": "DEF", "@id": 2, "n": "DEF"}]}, {"edges": [{"i": "interacts-with", "s": 1, "@id": 0, "t": 2}, {"i": "neighbor-of", "s": 2, "@id": 1, "t": 0}]}, {"networkAttributes": [{"v": "Pandas Upload", "d": "string", "n": "name"}]}, {"edgeAttributes": [{"v": "Edge property 1", "po": 0, "n": "EdgeProp"}, {"v": "Edge property 2", "po": 1, "n": "EdgeProp"}]}, {"metaData": [{"elementCount": 1, "version": "1.0", "consistencyGroup": 1, "name": "networkAttributes"}, {"idCounter": 2, "elementCount": 3, "version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"idCounter": 1, "element

# Create using network hosted on NDEx server

In [2]:
niceCx = NiceCXNetwork(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')
print(niceCx)


http://public.ndexbio.org/v2/network/f1dd6cc3-0007-11e6-b550-06603eb7f303/aspect
[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"elementCount": 659, "consistencyGroup": null, "name": "edgeAttributes"}, {"elementCount": 1, "consistencyGroup": null, "name": "subNetworks"}, {"elementCount": 2, "consistencyGroup": null, "name": "networkRelations"}, {"elementCount": 1, "lastUpdate": 1494530046623, "version": "1.0", "consistencyGroup": 1, "name": "provenanceHistory"}, {"elementCount": 11, "consistencyGroup": null, "name": "networkAttributes"}, {"elementCount": 778, "consistencyGroup": null, "name": "nodeAttributes"}, {"idCounter": 212, "elementCount": 37, "version": "1.0", "consistencyGroup": 1, "name": "edges"}, {"elementCount": 1, "consistencyGroup": null, "name": "cyViews"}, {"elementCount": 36, "consistencyGroup": null, "name": "cartesianLayout"}, {"idCounter": 212, "elementCount": 36, "version": "1.0", "consistencyGroup": 1, "name": "nodes"}, {"elementCount":